# Advent of Code, 9th Dec 2024

## Hoof It - Part 1

You all arrive at a Lava Production Facility on a floating island in the sky. As the others begin to search the massive industrial complex, you feel a small nose boop your leg and look down to discover a reindeer wearing a hard hat.

The reindeer is holding a book titled "Lava Island Hiking Guide". However, when you open the book, you discover that most of it seems to have been scorched by lava! As you're about to ask how you can help, the reindeer brings you a blank **topographic map** of the surrounding area (your puzzle input) and looks up at you excitedly.

Perhaps you can help fill in the missing hiking trails?

The topographic map indicates the **height** at each position using a scale from 0 (lowest) to 9 (highest). For example:

```
0123
1234
8765
9876
```

Based on un-scorched scraps of the book, you determine that a good hiking trail is **as long as possible** and has an **even, gradual, uphill slope**. For all practical purposes, this means that a **hiking trail** is any path that starts at height 0, ends at height 9, and always increases by a height of exactly 1 at each step. Hiking trails never include diagonal steps - only up, down, left, or right (from the perspective of the map).

You look up from the map and notice that the reindeer has helpfully begun to construct a small pile of pencils, markers, rulers, compasses, stickers, and other equipment you might need to update the map with hiking trails.

A **trailhead** is any position that starts one or more hiking trails - here, these positions will always have height 0. Assembling more fragments of pages, you establish that a trailhead's **score** is the number of 9-height positions reachable from that trailhead via a hiking trail. In the above example, the single trailhead in the top left corner has a score of 1 because it can reach a single 9 (the one in the bottom left).

This trailhead has a score of 2:

```
...0...
...1...
...2...
6543456
7.....7
8.....8
9.....9
```

(The positions marked . are impassable tiles to simplify these examples; they do not appear on your actual topographic map.)

This trailhead has a score of 4 because every 9 is reachable via a hiking trail except the one immediately to the left of the trailhead:

```
..90..9
...1.98
...2..7
6543456
765.987
876....
987....
```

This topographic map contains **two** trailheads; the trailhead at the top has a score of 1, while the trailhead at the bottom has a score of 2:

```
10..9..
2...8..
3...7..
4567654
...8..3
...9..2
.....01
```

Here's a larger example:

```
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
```

This larger example has 9 trailheads. Considering the trailheads in reading order, they have scores of 5, 6, 5, 3, 1, 3, 5, 3, and 5. Adding these scores together, the sum of the scores of all trailheads is **36**.

The reindeer gleefully carries over a protractor and adds it to the pile. **What is the sum of the scores of all trailheads on your topographic map?**

### Read in the data

In [57]:
#!F#
let inputFilePath = "aoc-2024-12-10-puzzle-input.txt"

Read the lines from the file.

In [58]:
open System.IO

let lines: string list = File.ReadAllLines(inputFilePath) |> Array.toList
lines.Length

60

In [59]:
let minLineLength = lines |> List.map (fun line -> line.Length) |> List.min
let maxLineLength = lines |> List.map (fun line -> line.Length) |> List.max
(minLineLength, maxLineLength)

Item1,60
Item2,60


So the input is a 60*60 square of characters (digits).

### Solve the problem

We need to convert the lines into a set of obstacle positions, and also the starting position of the guard.  We will mark the lower-left-hand corner as (0,0), so we will need to reverse the lines so that we start processing the lines from row 0.

In [60]:
let N = lines.Length
N

60

In [61]:
let identity n = n

let ch0 = int '0'
let numberedRows = Seq.zip (Seq.initInfinite identity) (lines |> List.map (fun line -> line |> Seq.toList |> List.map (fun ch -> (int ch) - ch0)) |> List.rev) |> Seq.toList
numberedRows[0]

(0, [3; 2; 3; ... ]) Item1 0 Item2 [ 3, 2, 3, 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6 ... (more) ] HeadOrDefault 3 TailOrNull [ 2, 3, 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7 ... (more) ] HeadOrDefault 2 TailOrNull [ 3, 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8 ... (more) ] HeadOrDefault 3 TailOrNull [ 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7 ... (more) ] HeadOrDefault 1 TailOrNull [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] Head 2 Tail [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] (values) [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] Head 1 Tail [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] Head 2 Tail [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] (values) [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] (values) [ 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7 ... (more) ] Head 3 Tail [ 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7 ... (more) ] HeadOrDefault 1 TailOrNull [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] Head 2 Tail [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] (values) [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] Head 1 Tail [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] Head 2 Tail [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] (values) [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] (values) [ 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7 ... (more) ] (values) [ 3, 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8 ... (more) ] Head 2 Tail [ 3, 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8 ... (more) ] HeadOrDefault 3 TailOrNull [ 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7 ... (more) ] HeadOrDefault 1 TailOrNull [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] Head 2 Tail [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] (values) [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] Head 1 Tail [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] Head 2 Tail [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] (values) [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] (values) [ 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7 ... (more) ] Head 3 Tail [ 1, 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7 ... (more) ] HeadOrDefault 1 TailOrNull [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] Head 2 Tail [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] (values) [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] Head 1 Tail [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] Head 2 Tail [ 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2, 1 ... (more) ] (values) [ 2, 7, 6, 5, 6, 1, 0, 3, 8, 9, 8, 2, 3, 4, 5, 6, 7, 8, 7, 2 ..

In [62]:
let buildTopographicMap (numberedRows: (int * int list) list) = (seq{
    for (row, colVals) in numberedRows do
        let indexColValues = Seq.zip (Seq.initInfinite identity) colVals |> Seq.toList
        for (col, value) in indexColValues do
            yield ((row, col), value)
} |> Map.ofSeq)

In [63]:
let topographyMap = buildTopographicMap numberedRows
topographyMap

key value (0, 0) Item1 0 Item2 0 3 (0, 1) Item1 0 Item2 1 2 (0, 2) Item1 0 Item2 2 3 (0, 3) Item1 0 Item2 3 1 (0, 4) Item1 0 Item2 4 2 (0, 5) Item1 0 Item2 5 7 (0, 6) Item1 0 Item2 6 6 (0, 7) Item1 0 Item2 7 5 (0, 8) Item1 0 Item2 8 6 (0, 9) Item1 0 Item2 9 1 (0, 10) Item1 0 Item2 10 0 (0, 11) Item1 0 Item2 11 3 (0, 12) Item1 0 Item2 12 8 (0, 13) Item1 0 Item2 13 9 (0, 14) Item1 0 Item2 14 8 (0, 15) Item1 0 Item2 15 2 (0, 16) Item1 0 Item2 16 3 (0, 17) Item1 0 Item2 17 4 (0, 18) Item1 0 Item2 18 5 (0, 19) Item1 0 Item2 19 6 ... (more)

In [64]:
let trailheads (topographyMap: Map<(int * int), int>) = (seq{
    for (row, col) in topographyMap.Keys do
        if topographyMap.[(row, col)] = 0 then
            yield (row, col)
} |> Seq.toList)

In [65]:
let trailheadspt1 = trailheads topographyMap
trailheadspt1

[ (0, 10), (0, 25), (0, 30), (0, 32), (0, 44), (0, 51), (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0) ... (more) ] HeadOrDefault (0, 10) Item1 0 Item2 10 TailOrNull [ (0, 25), (0, 30), (0, 32), (0, 44), (0, 51), (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12) ... (more) ] HeadOrDefault (0, 25) Item1 0 Item2 25 TailOrNull [ (0, 30), (0, 32), (0, 44), (0, 51), (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34) ... (more) ] HeadOrDefault (0, 30) Item1 0 Item2 30 TailOrNull [ (0, 32), (0, 44), (0, 51), (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34), (3, 36) ... (more) ] HeadOrDefault (0, 32) Item1 0 Item2 32 TailOrNull [ (0, 44), (0, 51), (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34), (3, 36), (3, 45) ... (more) ] HeadOrDefault (0, 44) Item1 0 Item2 44 TailOrNull [ (0, 51), (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34), (3, 36), (3, 45), (4, 9) ... (more) ] HeadOrDefault (0, 51) TailOrNull [ (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34), (3, 36), (3, 45), (4, 9), (4, 36) ... (more) ] Head (0, 51) Tail [ (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34), (3, 36), (3, 45), (4, 9), (4, 36) ... (more) ] (values) index value 0 (0, 51) 1 (0, 55) 2 (1, 3) 3 (1, 16) 4 (1, 22) 5 (1, 27) 6 (1, 28) 7 (1, 34) 8 (1, 39) 9 (1, 59) 10 (2, 1) 11 (2, 16) 12 (2, 17) 13 (2, 50) 14 (3, 0) 15 (3, 12) 16 (3, 34) 17 (3, 36) 18 (3, 45) 19 (4, 9) ... (more) Head (0, 44) Item1 0 Item2 44 Tail [ (0, 51), (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34), (3, 36), (3, 45), (4, 9) ... (more) ] HeadOrDefault (0, 51) TailOrNull [ (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34), (3, 36), (3, 45), (4, 9), (4, 36) ... (more) ] Head (0, 51) Tail [ (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34), (3, 36), (3, 45), (4, 9), (4, 36) ... (more) ] (values) index value 0 (0, 51) 1 (0, 55) 2 (1, 3) 3 (1, 16) 4 (1, 22) 5 (1, 27) 6 (1, 28) 7 (1, 34) 8 (1, 39) 9 (1, 59) 10 (2, 1) 11 (2, 16) 12 (2, 17) 13 (2, 50) 14 (3, 0) 15 (3, 12) 16 (3, 34) 17 (3, 36) 18 (3, 45) 19 (4, 9) ... (more) (values) index value 0 (0, 44) Item1 0 Item2 44 1 (0, 51) Item1 0 Item2 51 2 (0, 55) Item1 0 Item2 55 3 (1, 3) Item1 1 Item2 3 4 (1, 16) Item1 1 Item2 16 5 (1, 22) Item1 1 Item2 22 6 (1, 27) Item1 1 Item2 27 7 (1, 28) Item1 1 Item2 28 8 (1, 34) Item1 1 Item2 34 9 (1, 39) Item1 1 Item2 39 10 (1, 59) Item1 1 Item2 59 11 (2, 1) Item1 2 Item2 1 12 (2, 16) Item1 2 Item2 16 13 (2, 17) Item1 2 Item2 17 14 (2, 50) Item1 2 Item2 50 15 (3, 0) Item1 3 Item2 0 16 (3, 12) Item1 3 Item2 12 17 (3, 34) Item1 3 Item2 34 18 (3, 36) Item1 3 Item2 36 19 (3, 45) Item1 3 Item2 45 ... (more) Head (0, 32) Item1 0 Item2 32 Tail [ (0, 44), (0, 51), (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34), (3, 36), (3, 45) ... (more) ] HeadOrDefault (0, 44) Item1 0 Item2 44 TailOrNull [ (0, 51), (0, 55), (1, 3), (1, 16), (1, 22), (1, 27), (1, 28), (1, 34), (1, 39), (1, 59), (2, 1), (2, 16), (2, 17), (2, 50), (3, 0), (3, 12), (3, 34), (3, 36), (3, 45), (4, 9) ... (more) ] 

In [66]:
trailheadspt1.Length

327

In [67]:
type Direction = | Up | Right | Down | Left
let allDirections = [Up; Right; Down; Left]

In [68]:
let move (row, col) direction: (int*int) option = 
    match direction with
    | Up -> if (row > 0) then Some((row - 1, col)) else None
    | Right -> if (col < N-1) then Some((row, col + 1)) else None
    | Down -> if (row < N-1) then Some((row + 1, col)) else None
    | Left -> if (col > 0) then Some((row, col - 1)) else None

In [69]:
let mavVal = 9

let rec buildPaths (start: int*int): (int*int) list list = (seq {
    let startValue = topographyMap.[start]
    for direction in allDirections do
        let next = move start direction
        match next with
        | Some(next) -> 
            let nextValue = topographyMap.[next]
            if nextValue = startValue + 1 then
                if nextValue = mavVal then
                    yield [start; next]
                else
                    yield! buildPaths next |> List.map (fun path -> start::path)
        | None -> yield []
} |> Seq.toList |> List.filter (fun path -> not path.IsEmpty))

Let's quickly sanity check that our paths run from zero to nine, as they should.

In [70]:
for path in buildPaths trailheadspt1.[0] do
    printfn "%A" (path |> List.map (fun (row, col) -> topographyMap.[(row, col)]))

[0; 1; 2; 3; 4; 5; 6; 7; 8; 9]
[0; 1; 2; 3; 4; 5; 6; 7; 8; 9]


In [71]:
let trailheadScore (trailhead: int*int) = 
    buildPaths trailhead |> List.map (List.last) |> Set.ofList |> Set.count

In [72]:
trailheadspt1 |> List.map trailheadScore

[ 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1 ... (more) ] HeadOrDefault 1 TailOrNull [ 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1 ... (more) ] HeadOrDefault 2 TailOrNull [ 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2 ... (more) ] HeadOrDefault 1 TailOrNull [ 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2 ... (more) ] HeadOrDefault 1 TailOrNull [ 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] HeadOrDefault 1 TailOrNull [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] Head 1 Tail [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] (values) [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] Head 2 Tail [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] HeadOrDefault 1 TailOrNull [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] Head 1 Tail [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] (values) [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] (values) [ 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2 ... (more) ] Head 1 Tail [ 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] HeadOrDefault 1 TailOrNull [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] Head 1 Tail [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] (values) [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] Head 2 Tail [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] HeadOrDefault 1 TailOrNull [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] Head 1 Tail [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] (values) [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] (values) [ 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2 ... (more) ] (values) [ 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2 ... (more) ] Head 1 Tail [ 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2 ... (more) ] HeadOrDefault 1 TailOrNull [ 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] HeadOrDefault 1 TailOrNull [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] Head 1 Tail [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] (values) [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] Head 2 Tail [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] HeadOrDefault 1 TailOrNull [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] Head 1 Tail [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] (values) [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] (values) [ 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2 ... (more) ] Head 1 Tail [ 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2 ... (more) ] HeadOrDefault 2 TailOrNull [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] HeadOrDefault 1 TailOrNull [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] Head 1 Tail [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] (values) [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] Head 2 Tail [ 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3 ... (more) ] HeadOrDefault 1 TailOrNull [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] Head 1 Tail [ 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2 ... (more) ] (value

In [73]:
let totalScorept1 = trailheadspt1 |> List.map trailheadScore |> List.sum
totalScorept1

811

## Hoof It - Part 2

The reindeer spends a few minutes reviewing your hiking trail map before realizing something, disappearing for a few minutes, and finally returning with yet another slightly-charred piece of paper.

The paper describes a second way to measure a trailhead called its **rating**. A trailhead's rating is the **number of distinct hiking trails** which begin at that trailhead. For example:

```
.....0.
..4321.
..5..2.
..6543.
..7..4.
..8765.
..9....
```

The above map has a single trailhead; its rating is 3 because there are exactly three distinct hiking trails which begin at that position:

```
.....0.   .....0.   .....0.
..4321.   .....1.   .....1.
..5....   .....2.   .....2.
..6....   ..6543.   .....3.
..7....   ..7....   .....4.
..8....   ..8....   ..8765.
..9....   ..9....   ..9....
```

Here is a map containing a single trailhead with rating 13:

```
..90..9
...1.98
...2..7
6543456
765.987
876....
987....
```

This map contains a single trailhead with rating 227 (because there are 121 distinct hiking trails that lead to the 9 on the right edge and 106 that lead to the 9 on the bottom edge):

```
012345
123456
234567
345678
4.6789
56789.
```

Here's the larger example from before:

```
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
```

Considering its trailheads in reading order, they have ratings of 20, 24, 10, 4, 1, 4, 5, 8, and 5. The sum of all trailhead ratings in this larger example topographic map is **81**.

You're not sure how, but the reindeer seems to have crafted some tiny flags out of toothpicks and bits of paper and is using them to mark trailheads on your topographic map. **What is the sum of the ratings of all trailheads?**

### Solve the problem

In [77]:
let trailheadRating (trailhead: int*int) = 
    buildPaths trailhead |> List.length

In [78]:
let totalScorept2 = trailheadspt1 |> List.map trailheadRating |> List.sum
totalScorept2

1794